# Pilih menu " **Runtime** " lalu " **Run All** " => " ** Run Anyway** "

# Untuk menjalankan suatu cell (setelah melakukan perubahan) tekan tombol "CTRL + Enter"

In [ ]:
!pip install folium
!pip install pyldavis
!wget https://tau-data.id/d/taudata_min.py
!wget https://tau-data.id/d/stunt1.pkl
!wget https://tau-data.id/d/stunt2.pkl
import warnings; warnings.simplefilter('ignore')
import taudata_min as tau, folium, pickle
from folium import plugins
import pandas as pd, seaborn as sns; sns.set()
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import pyLDAvis, pyLDAvis.sklearn; pyLDAvis.enable_notebook()
from folium.plugins import HeatMap

In [ ]:
fData1 = 'stunt1.pkl'
fData2 = 'stunt2.pkl'
'2018-10-01', '2019-11-01'

In [ ]:
# Loading Data for interactive visualizations
with open(fData1, 'rb') as f:
    places, tweets, Dt, koordinat, txtPlaces, lokasi = pickle.load(f)

with open(fData2, 'rb') as f:
    webpages, webCategory, df_w, Dw  = pickle.load(f)

In [ ]:
tweets.head()

In [ ]:
tweets.info()

In [ ]:
df = pd.DataFrame({'Lokasi':lokasi})
plt.figure(figsize=(12,10))
sns.set(style='darkgrid')
sns.countplot(y = 'Lokasi', data = df, order = df['Lokasi'].value_counts().index)
plt.show()

In [ ]:
print(places)

In [ ]:
txtPlaces['brebes']

In [ ]:
tolerance = 10**-4
latID, lonID, count = -0.789275, 113.921, 1
lat, lon = [], []
for i, r in tqdm(tweets.iterrows()):
    if (r.lat!=0 and r.lon!=0) and (abs(r.lat-latID)>tolerance and abs(r.lon-lonID)>tolerance):
        lat.append(r.lat)
        lon.append(r.lon) 
count = [1]*len(lat)
df = pd.DataFrame({'lat':lat, 'lon':lon, 'count':count})
df.head()

In [ ]:
base_map = tau.generateBaseMap()
HeatMap(data=df[['lat', 'lon', 'count']].groupby(['lat', 'lon']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)
base_map

In [ ]:
#df = pd.DataFrame({'Sentiment':sentiments})
c = tweets['sentiment'].value_counts()
values = c.values
colors = ['green', 'red', 'blue']
labels = c.index
explode = (0.2, 0, 0)
plt.figure(figsize=(12,10))

plt.pie(values, colors=colors, labels= values, explode=explode, shadow=True, textprops={'fontsize': 20})
plt.title('Sentimen Pengguna twitter 1 Okt 2018 - Nov 2019 tentang isu Stunting di Indonesia\n\n')
plt.legend(labels,loc='best')
plt.show()

In [ ]:
# Mari kita dalami lebih jauh Topic Pembicaraan ini
k = 4 # Jumlah Topik ... menurut CV yg sudah dilakukan 4 adalah yg optimal. Tapi silahkan eksplore utk jumlah topik lainnya
Docs_t = [' '.join(d) for d in Dt]
tf_t, tm_t, vec_t = tau.getTopics(Docs_t, n_topics=k, Top_Words=10)

In [ ]:
pyLDAvis.sklearn.prepare(tf_t, tm_t, vec_t)  

# Analisa pada data Website

In [ ]:
webpages.head()

In [ ]:
webpages.info()

In [ ]:
plt.figure(figsize=(12,10))
sns.set(style='darkgrid')
sns.countplot(y = 'Website', data = df_w, order = df_w['Website'].value_counts().index)
plt.show()

In [ ]:
# Mari kita dalami lebih jauh Topic Pembicaraan ini
k = 3
Docs_w = [' '.join(d) for d in Dw]
tf_w, tm_w, vec_w = tau.getTopics(Docs_w, n_topics=k, Top_Words=10)

In [ ]:
pyLDAvis.sklearn.prepare(tf_w, tm_w, vec_w)  

# Social Network Analysis

In [ ]:
Tweets = []
for i, r in tweets.iterrows():
    Tweets.append([r.username, r.tweet])
    
G = tau.Graph(Tweets)

In [ ]:
# Let's now examine, who are the most "important" users in this Graph?
"""
silahkan pilih dari beberapa metode ini
degree = important karena banyak yang mention atau dimention
closeness = important karena jaraknya dengan semua user lain. Informasi akan cepat menyebar melalui orang ini
eigen = important karena user tersebut ber-degree tinggi atau degree rendah tapi mempengaruhi orang yg degree tinggi (kuda hitam)
betweeness = user-user ini important karena bisa mencegah suatu informasi menyebar lebih jauh

N = jumlah important user yang ingin ditampilkan
"""
Gt = tau.Centrality(G, N=5, method='degree')

In [ ]:
tau.drawGraph(Gt, Label = True)

# Pemisahan Text dan Topic Modelling Berdasarkan Kategori Web

In [ ]:
webCategory.head()

In [ ]:
Cats = list(set(webCategory.category))
Cats

In [ ]:
# Silahkan rubah 'cat' menjadi = ['NGO', 'Pemerintah Daerah', 'Pemerintah Pusat', 'Portal']
# Lalu jalankan cell ini : "tombol ctrl+Enter"
cat, k = 'NGO', 3
Docs = []
for i, r in tqdm(webCategory.iterrows()):
    if r.category == cat:
        Docs.append(r.content)
        
tf, tm, vec = tau.getTopics(Docs, n_topics=k, Top_Words=10)

In [ ]:
# Visualisasi topic dari cell diatas (tekan tombol ctrl+Enter, lalu tunggu ... biasanya agak lama utk graphics muncul)
pyLDAvis.sklearn.prepare(tf, tm, vec)  

# Pemisahan, preprocessing, dan penyimpanan Text Tweets berdasarkan Lokus Penelitian dan Sentiment untuk dilakukan Text Analytics.

# Text Analytics dapat dilakukan dengan cara:

1. Buka https://voyant-tools.org/
2. Upload berbagai file Text, misal Tweet_solok.txt, Web_NGO.txt, dsb ==> file-file ini sudah dikirim melalui email dan WA
3. Pilih berbagai visualisasi yang dibutuhkan: WordCloud, WordLinks, Word Tree, dsb